In [ ]:
import os
import pydicom
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
import csv
from PIL import Image
import numpy as np
import glob

In [ ]:
dicom_directory = 'D:\\CTH_archive\\DICOM'
all_files_metadata = []

# Dictionaries to track patient files
patients_colored_files = {}  # PatientID -> List of 'viz tmax colored' file paths
patients_tmax_files = {}     # PatientID -> List of 'viz tmax' file paths

total_files = sum([len(files) for r, d, files in os.walk(dicom_directory) if any(file.endswith('.dcm') for file in files)])

with tqdm(total=total_files, desc="Processing DICOM Files") as pbar:
    for root, dirs, files in os.walk(dicom_directory):
        for file in files:
            if file.endswith('.dcm'):
                file_path = os.path.join(root, file)
                try:
                    dicom = pydicom.dcmread(file_path, stop_before_pixels=True)
                    metadata = {tag: getattr(dicom, tag, '') for tag in dicom.dir()}
                    metadata['FilePath'] = file_path
                    all_files_metadata.append(metadata)

                    metadata_str_lower = {k: str(v).lower() for k, v in metadata.items()}
                    metadata_combined_str = ' '.join(metadata_str_lower.values())

                    patient_id = metadata.get('PatientID')
                    if "viz tmax colored" in metadata_combined_str:
                        patients_colored_files.setdefault(patient_id, []).append(file_path)
                    elif "viz tmax" in metadata_combined_str:
                        patients_tmax_files.setdefault(patient_id, []).append(file_path)

                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
                pbar.update(1)

# Determine final file list, prioritizing 'viz tmax colored'
matching_files = []
for patient_id, file_paths in patients_colored_files.items():
    matching_files.extend(file_paths)

for patient_id, file_paths in patients_tmax_files.items():
    if patient_id not in patients_colored_files:
        matching_files.extend(file_paths)

# Print or process the matching files
for file in matching_files:
    print(f"Matching file: {file}")


In [ ]:
def count_unique_mrn(file_paths):
    """
    Count unique MRNs in a list of DICOM file paths.

    :param file_paths: List of file paths.
    :return: The count of unique MRNs.
    """
    unique_mrns = set()

    for path in file_paths:
        # Split the path and extract the MRN segment
        parts = path.split(os.sep)

        mrn = parts[3]
        unique_mrns.add(mrn)
            

    return len(unique_mrns)


unique_mrn_count = count_unique_mrn(matching_files)
print(f"Unique MRN count: {unique_mrn_count}")

In [ ]:
def list_unique_mrn(file_paths):
    """
    Count unique MRNs in a list of DICOM file paths.

    :param file_paths: List of file paths.
    :return: The count of unique MRNs.
    """
    unique_mrns = set()

    for path in file_paths:
        # Split the path and extract the MRN segment
        parts = path.split(os.sep)

        mrn = parts[3]
        unique_mrns.add(mrn)
            

    return unique_mrns


mrn_list = list_unique_mrn(matching_files)
print(f"Unique MRN count: {mrn_list}")

In [ ]:
with open('CTP.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for path in matching_files:
        writer.writerow([path]) 

In [ ]:
import os
import pydicom
from tqdm import tqdm
search_terms = {"Head  5.0  J45s  1", "AXIAL", "AXIAL (Stroke Alert)", "Head  5.0  MPR  ax", 
    "AXIAL STND", "Axial Head 5.0", "Soft", "Brain Brain", 
    "ROUTINE HEAD", "AX SOFT", "AX5ST."}


def search_dicom_files(directory, terms):
    matching_files = []

    # Create a list of all DICOM files
    dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(directory) 
                   for file in files if file.endswith(".dcm")]


    for file_path in tqdm(dicom_files, desc="Processing DICOM files"):
        try:
            dicom_file = pydicom.dcmread(file_path)
            if any(term in dicom_file.SeriesDescription for term in terms):
                matching_files.append(file_path)
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

    return matching_files


matching_file_paths = search_dicom_files("D:\\CTH_archive\\DICOM", search_terms)
print(matching_file_paths)


In [ ]:
import os
import pydicom
from tqdm import tqdm
search_terms = {"VPCT Perfusion 5.0 H20f, CTP, CT PERFUSION, CTP 5.0 Hr38"}


def search_dicom_files(directory, terms):
    matching_files = []

    # Create a list of all DICOM files
    dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(directory) 
                   for file in files if file.endswith(".dcm")]


    for file_path in tqdm(dicom_files, desc="Processing DICOM files"):
        try:
            dicom_file = pydicom.dcmread(file_path)
            if any(term in dicom_file.SeriesDescription for term in terms):
                matching_files.append(file_path)
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

    return matching_files


matching_file_paths = search_dicom_files("D:\\CTH_archive\\DICOM", search_terms)
print(matching_file_paths)


In [ ]:
with open('CTH.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for path in matching_file_paths:
        writer.writerow([path]) 

In [ ]:
unique_mrn_count = count_unique_mrn(matching_file_paths )
mathcing_files_list =  list_unique_mrn(matching_file_paths)
print(f"Unique MRN count: {unique_mrn_count}")
print(f"mathcing_files_list: {mathcing_files_list}")

In [ ]:
with open('CTP.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in mathcing_files_list:
        name, number = item.rsplit(' ', 1)  # Split each item into name and number
        writer.writerow([name, number]) 

In [ ]:
mathcing_files_list ^ set(os.listdir("D:\\CTH_archive\\DICOM"))

In [ ]:

# Directory containing DICOM files
dicom_dir = r"D:\CTH_archive\DICOM\KUHNER_ROSEMARY 4716150\2020-09-11 214748"

# List to hold DICOM descriptions
dicom_descriptions = []

# Loop through all files in the directory
for filename in os.listdir(dicom_dir):
    if filename.endswith(".dcm"):
        filepath = os.path.join(dicom_dir, filename)
        ds = pydicom.dcmread(filepath)
        
        # Extract description and add it to the list
        description = ds.get("SeriesDescription", "No description available")
        dicom_descriptions.append(description)

# Print out all descriptions
for desc in dicom_descriptions:
    print(desc)


In [ ]:
folder_path = r'D:\CTH_archive\DICOM\KUHNER_ROSEMARY 4716150\2020-09-11 214748'
rows_list = []
columns_list = []

for file in os.listdir(folder_path):
    if file.endswith(".dcm"):  # Checking for DICOM files
        file_path = os.path.join(folder_path, file)
        dicom_data = pydicom.dcmread(file_path)

        # Extracting rows and columns
        rows = dicom_data.Rows
        columns = dicom_data.Columns

        rows_list.append(rows)
        columns_list.append(columns)

# Plotting histograms
if rows_list and columns_list:
    plt.figure(figsize=(12, 6))

    # Histogram for Rows
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    plt.hist(rows_list, bins=10, color='blue', edgecolor='black')
    plt.title('Histogram of DICOM Image Rows')
    plt.xlabel('Rows')
    plt.ylabel('Frequency')

    # Histogram for Columns
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
    plt.hist(columns_list, bins=10, color='green', edgecolor='black')
    plt.title('Histogram of DICOM Image Columns')
    plt.xlabel('Columns')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()
else:
    print("No DICOM files found in the specified folder.")


In [ ]:
folder_path = r'D:\CTH_archive\DICOM\KUHNER_ROSEMARY 4716150\2020-09-11 220939'
rows_list = []
columns_list = []

for file in os.listdir(folder_path):
    if file.endswith(".dcm"):  # Checking for DICOM files
        file_path = os.path.join(folder_path, file)
        dicom_data = pydicom.dcmread(file_path)

        # Extracting rows and columns
        rows = dicom_data.Rows
        columns = dicom_data.Columns

        rows_list.append(rows)
        columns_list.append(columns)

# Plotting histograms
if rows_list and columns_list:
    plt.figure(figsize=(12, 6))

    # Histogram for Rows
    plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    plt.hist(rows_list, bins=10, color='blue', edgecolor='black')
    plt.title('Histogram of DICOM Image Rows')
    plt.xlabel('Rows')
    plt.ylabel('Frequency')

    # Histogram for Columns
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
    plt.hist(columns_list, bins=10, color='green', edgecolor='black')
    plt.title('Histogram of DICOM Image Columns')
    plt.xlabel('Columns')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()
else:
    print("No DICOM files found in the specified folder.")


In [ ]:
import csv
import os
import pydicom
from PIL import Image
import numpy as np

# Base directory for the DICOM files and the new cropped folder
base_directory = r'D:\CTH_archive'
dicom_directory = os.path.join(base_directory, 'DICOM')
cropped_directory = os.path.join(base_directory, 'cropped')

# Path to the CSV file
csv_file_path = r'C:\Users\nprim\Desktop\CTP.csv'

# Define the cropping coordinates
x1, y1, x2, y2 = 50, 35, 235, 235  # The rectangle coordinates

# Read the file paths from the CSV
dicom_file_paths = []
with open(csv_file_path, mode='r', newline='', encoding='utf-8-sig') as file:
    reader = csv.reader(file)
    for row in reader:
        dicom_file_paths.extend(row)  # Assuming each row contains one file path

# Process each DICOM file
for file_path in dicom_file_paths:
    try:
        # Read the DICOM file
        dicom_data = pydicom.dcmread(file_path)
        image_array = dicom_data.pixel_array

        # Convert to a format that PIL can understand
        if len(image_array.shape) == 2:  # Grayscale
            image = Image.fromarray(image_array).convert('L')
        elif len(image_array.shape) == 3:  # RGB
            image = Image.fromarray(image_array)

        # Convert to JPG
        jpg_file_path = file_path.replace('.dcm', '.jpg')
        image.save(jpg_file_path)

        # Open the JPG image and crop it
        jpg_image = Image.open(jpg_file_path)
        cropped_image = jpg_image.crop((x1, y1, x2, y2))

        # Replicate the folder structure in the cropped directory
        relative_path = os.path.relpath(jpg_file_path, dicom_directory)
        cropped_file_path = os.path.join(cropped_directory, relative_path)
        os.makedirs(os.path.dirname(cropped_file_path), exist_ok=True)

        # Save the cropped image
        cropped_image.save(cropped_file_path)

        print(f"Cropped image saved to {cropped_file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
# Base directory where .dcm files need to be deleted
base_directory = r'D:\CTH_archive\cropped'

# Recursively find all .dcm files
dcm_files = glob.glob(os.path.join(base_directory, '**', '*.dcm'), recursive=True)

# Delete each found .dcm file
for file_path in dcm_files:
    try:
        os.remove(file_path)
        print(f"Deleted file: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

print("Deletion of .dcm files complete.")


In [ ]:

def apply_modality_lut(image_array, dicom_data):
    """
    Apply Modality LUT (Rescale Slope and Intercept).
    """
    rescale_slope = dicom_data.RescaleSlope if 'RescaleSlope' in dicom_data else 1
    rescale_intercept = dicom_data.RescaleIntercept if 'RescaleIntercept' in dicom_data else 0
    return image_array * rescale_slope + rescale_intercept

def apply_windowing(image_array, window_center, window_width):
    """
    Apply windowing to the DICOM image array.
    """
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    windowed_image = np.clip(image_array, img_min, img_max)
    windowed_image = ((windowed_image - img_min) / (img_max - img_min)) * 255.0
    return windowed_image.astype(np.uint8)

# Paths
csv_file_path = r'C:\Users\nprim\Desktop\CTH.csv'
base_directory = r'D:\CTH_archive\DICOM'
target_directory = r'D:\CTH_archive\CTH_JPG'

# Read file paths from CSV
file_paths = []
with open(csv_file_path, mode='r', newline='', encoding='utf-8-sig') as file:
    reader = csv.reader(file)
    for row in reader:
        file_paths.extend(row)

# Process each file
for file_path in file_paths:
    try:
        # Read DICOM file
        dicom_data = pydicom.dcmread(file_path)
        image_array = apply_modality_lut(dicom_data.pixel_array, dicom_data)

        # Apply windowing (adjust these values as needed)
        #window_center, window_width = 40, 80  # Example values; adjust based on your DICOM data
        #if 'WindowCenter' in dicom_data and 'WindowWidth' in dicom_data:
        #    window_center, window_width = int(dicom_data.WindowCenter), int(dicom_data.WindowWidth)

            # Handling WindowCenter and WindowWidth
        if 'WindowCenter' in dicom_data and 'WindowWidth' in dicom_data:
            window_center = dicom_data.WindowCenter
            window_width = dicom_data.WindowWidth

        # Check if WindowCenter or WindowWidth is a MultiValue and handle accordingly
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = int(window_center[0])  # Taking the first value as an example
        else:
            window_center = int(window_center)
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = int(window_width[0])  # Taking the first value as an example
        else:
            window_width = int(window_width)



        image_array = apply_windowing(image_array, window_center, window_width)

        # Convert to image and save as JPG
        image = Image.fromarray(image_array).convert('L')  # Convert to grayscale
        relative_path = os.path.relpath(file_path, base_directory)
        new_path = os.path.join(target_directory, relative_path)
        new_path = new_path.replace('.dcm', '.jpg')  # Change extension to .jpg
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        image.save(new_path, 'JPEG')
        print(f"Saved JPG file at {new_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image

def convert_jpg_to_nifti(jpg_dir, output_dir):
    for subdir, dirs, files in os.walk(jpg_dir):
        # Filter and sort JPG files
        jpg_files = sorted([f for f in files if f.endswith('.jpg')])

        # Skip directories without JPG files
        if not jpg_files:
            continue

        # Initialize an empty list to hold image data
        image_data = []

        # Read each image and append to the list
        for file in jpg_files:
            img_path = os.path.join(subdir, file)
            img = Image.open(img_path).convert('L')  # convert to grayscale if needed
            img_array = np.array(img)
            image_data.append(img_array)

        # Stack images into a 3D array
        image_3d = np.stack(image_data, axis=-1)

        # Create a NIfTI image (Assuming no header info, default affine)
        nifti_img = nib.Nifti1Image(image_3d, affine=np.eye(4))

        # Construct output file path
        subdir_name = os.path.basename(subdir)
        output_file = os.path.join(output_dir, subdir_name + '.nii')

        # Save the NIfTI image
        nib.save(nifti_img, output_file)
        print(f"Saved NIfTI image for {subdir_name}")

# Example usage
convert_jpg_to_nifti(r'D:\CTH_archive\CTP_CROPPED_JPG', r'D:\CTH_archive\CTP_NFTI')


In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image

def convert_jpg_to_nifti(jpg_dir, output_dir):
    for subdir, dirs, files in os.walk(jpg_dir):
        jpg_files = sorted([f for f in files if f.endswith('.jpg')])
        if not jpg_files:
            continue

        image_data = []
        for file in jpg_files:
            img_path = os.path.join(subdir, file)
            img = Image.open(img_path).convert('L')
            img_array = np.array(img)
            image_data.append(img_array)

        if not image_data:
            continue

        image_3d = np.stack(image_data, axis=-1)

        # Calculate relative path from jpg_dir and replace spaces with underscores
        relative_subdir = os.path.relpath(subdir, jpg_dir)
        relative_subdir = relative_subdir.replace(' ', '_')
        
        # Create output subdirectory path
        output_subdir = os.path.join(output_dir, relative_subdir)
        os.makedirs(output_subdir, exist_ok=True)

        # Construct output file path
        output_file_name = f'{os.path.basename(relative_subdir)}.nii'
        output_file = os.path.join(output_subdir, output_file_name)

        # Save the NIfTI image
        nifti_img = nib.Nifti1Image(image_3d, affine=np.eye(4))
        nib.save(nifti_img, output_file)
        print(f"Saved NIfTI image at {output_file}")

# Example usage
convert_jpg_to_nifti(r'D:\CTH_archive\CTP_CROPPED_JPG', r'D:\CTH_archive\CTP_NFTI')
